# 🧪 Generador de Datos Sucios para BI — Dataset Simulado Industria de Bebidas

Este script genera **datos sintéticos con inconsistencias controladas**, útiles para pruebas de **calidad de datos, validación ETL y modelos de limpieza**.  
El conjunto resultante simula un entorno real con errores comunes en las dimensiones y hechos de un modelo estrella.

---

## 📦 Librerías necesarias

```python
import pandas as pd
import numpy as np
import random
from faker import Faker
⚙️ Parámetros de configuración
python
Copiar código
# Porcentaje de registros con errores (0.0025 = 0.25%)
INCONSISTENCY_RATIO = 0.0025  

# Inicializador de Faker con localización en español (España)
fake = Faker('es_ES')
🧭 1. Generación de Dimensiones y Hechos
🗓️ generar_dim_tiempo(start_date, end_date)
Crea la Dimensión Tiempo con granularidad diaria, incluyendo:

Fecha_ID, Anio, Mes, Nombre_Mes, Numero_Semana, Trimestre, Estacionalidad_Factor

Simula errores de datos:

Fecha_ID nula o malformada (ID_ERRONEO, FECHA_INCORRECTA, etc.)

Inconsistencias en Nombre_Mes (mayúsculas, abreviaturas)

python
Copiar código
dim_tiempo = generar_dim_tiempo('2023-01-01', '2024-12-31')
🏬 generar_dim_tienda(num_tiendas)
Crea la Dimensión Tienda con las columnas:

Tienda_ID, Region, Formato_Tienda, Atractivo_Factor

Simula errores de:

Categorización (Region con tildes o mayúsculas)

Formatos (Formato_Tienda alternado con nombres como MiniMarket)

python
Copiar código
dim_tienda = generar_dim_tienda(50)
🧴 generar_dim_producto(num_productos)
Crea la Dimensión Producto con:

Producto_ID, Marca, Segmento, Sabor, Tipo_Envase, Tamano, Precio_Base, Demanda_Factor

Simula errores:

Outliers en precios (Precio_Base negativos)

Inconsistencias de capitalización en marcas

python
Copiar código
dim_producto = generar_dim_producto(100)
💰 generar_fact_ventas(dim_tiempo, dim_producto, dim_tienda, num_rows)
Genera la tabla de hechos Fact_Ventas con distribución uniforme mensual.
Incluye columnas:

Tienda_ID, Producto_ID, Fecha_ID, Distribucion_Numerica, Distribucion_Ponderada,
Precio, Out_Of_Stock_Flag, Ventas_Volumen, Ventas_Valor

Simula errores de datos:

Precio nulo (valores faltantes)

Ventas_Volumen con ceros (devoluciones o errores de carga)

python
Copiar código
fact_ventas = generar_fact_ventas(dim_tiempo, dim_producto, dim_tienda, 1000000)
📤 2. Función Principal de Exportación
🧾 generar_datos_sucio(...)
Genera y exporta todos los archivos CSV (dim_tiempo, dim_tienda, dim_producto, fact_ventas)
con datos sucios controlados.

Parámetros principales:

base_filename: prefijo del archivo

num_dias: rango de días a simular

num_tiendas, num_productos: cardinalidad de las dimensiones

num_rows_fact: número total de registros en la tabla de hechos

python
Copiar código
generar_datos_sucio(
    base_filename='datos_bi_sucio_bebidas',
    num_dias=730,
    num_tiendas=50,
    num_productos=100,
    num_rows_fact=1_000_000
)
🚀 Ejecución en modo principal
Ejemplo de ejecución completa para un dataset de 40 millones de filas (simulación masiva):

python
Copiar código
if __name__ == "__main__":
    generar_datos_sucio(
        base_filename='datos_bi_sucio_bebidas_40M',
        num_dias=730,
        num_tiendas=50,
        num_productos=100,
        num_rows_fact=40_000_000
    )
📊 Resultado Esperado
Archivo CSV	Filas (aprox.)	Contenido
datos_bi_sucio_bebidas_dim_tiempo.csv	730	Fechas diarias con errores en IDs
datos_bi_sucio_bebidas_dim_tienda.csv	50	Regiones y formatos inconsistentes
datos_bi_sucio_bebidas_dim_producto.csv	100	Productos con precios y nombres inconsistentes
datos_bi_sucio_bebidas_fact_ventas.csv	1,000,000+	Ventas simuladas con valores nulos o cero

🧹 Uso recomendado
Este generador puede servir para:

Simular flujos ETL con detección y corrección de errores

Entrenar modelos de validación y limpieza de datos

Crear escenarios de data governance y auditoría de calidad

💡 Consejo: Si vas a generar más de 5M de filas, exporta a parquet en lugar de CSV para mejorar el rendimiento.ar.
ar.
')
 import Faker


In [34]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import random
from faker import Faker

# Definición del ratio de inconsistencia para simular datos sucios
# 0.04 = 4% de los registros tendrán algún tipo de error
INCONSISTENCY_RATIO = 0.0025

# Inicializar Faker para generar datos falsos
fake = Faker('es_ES')

# ==============================================================================
# 1. Generación de Dimensiones y Hechos
# ==============================================================================

def generar_dim_tiempo(start_date, end_date):
    """
    Crea la tabla de Dimensión Tiempo con granularidad diaria e inyecta errores 
    controlados en la columna Fecha_ID (Nulos y Formato Incorrecto).
    """
    print("-> Generando Dim_Tiempo...")
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    dim_tiempo = pd.DataFrame({'Fecha': date_range})
    
    # Paso 1: Inicializamos Fecha_ID como string (object)
    dim_tiempo['Fecha_ID'] = dim_tiempo['Fecha'].dt.strftime('%Y%m%d').astype(str)
    
    dim_tiempo['Anio'] = dim_tiempo['Fecha'].dt.year
    dim_tiempo['Mes'] = dim_tiempo['Fecha'].dt.month
    dim_tiempo['Nombre_Mes'] = dim_tiempo['Fecha'].dt.month_name(locale='es_ES')
    dim_tiempo['Numero_Semana'] = dim_tiempo['Fecha'].dt.isocalendar().week.astype(int)
    dim_tiempo['Trimestre'] = dim_tiempo['Fecha'].dt.to_period('Q').astype(str)
    
    # Factor de estacionalidad basado en coseno (máximo en verano, mínimo en invierno)
    dim_tiempo['Estacionalidad_Factor'] = np.cos((dim_tiempo['Fecha'].dt.dayofyear - 180) * 2 * np.pi / 365)
    
    # --- INYECCIÓN DE ERRORES CONTROLADOS EN FECHA_ID (3-4% APROX) ---
    sample_size_id_error = int(len(dim_tiempo) * INCONSISTENCY_RATIO)
    error_indices = np.random.choice(dim_tiempo.index, size=sample_size_id_error, replace=False)
    
    # Dividir los errores en nulos y mal formateados
    half_error_size = sample_size_id_error // 2
    
    # PROBLEMA 1.1: Inyectar Valores Nulos (Aprox. 2%)
    null_indices = error_indices[:half_error_size]
    dim_tiempo.loc[null_indices, 'Fecha_ID'] = np.nan # Usamos np.nan para simular valores nulos
    print(f"   - Inyectados {len(null_indices)} valores nulos en Fecha_ID.")

    # PROBLEMA 1.2: Inyectar Mal Formateados (Texto o ID inventado, Aprox. 2%)
    misformat_indices = error_indices[half_error_size:]
    misformat_options = ['ID_ERRONEO', 'FECHA_INCORRECTA', '2024-X-01']
    for idx in misformat_indices:
        # Reemplazar con una cadena de error o texto no numérico
        dim_tiempo.loc[idx, 'Fecha_ID'] = random.choice(misformat_options)
    print(f"   - Inyectados {len(misformat_indices)} errores de formato en Fecha_ID.")
    # -------------------------------------------------------------------------
    
    # PROBLEMA 2: Inconsistencia en Nombre_Mes (Ej: abreviaturas, mayúsculas) - Mantenido
    sample_size_month = int(len(dim_tiempo) * INCONSISTENCY_RATIO)
    inconsistency_indices = np.random.choice(dim_tiempo.index, size=sample_size_month, replace=False)
    
    for idx in inconsistency_indices:
        original_name = dim_tiempo.loc[idx, 'Nombre_Mes']
        if pd.notnull(original_name): # Asegurar que no trabajamos sobre un nan
            if random.random() < 0.5:
                dim_tiempo.loc[idx, 'Nombre_Mes'] = original_name.upper() # Todo mayúsculas
            else:
                dim_tiempo.loc[idx, 'Nombre_Mes'] = original_name[:3].upper() + '.' # Abreviatura
    
    return dim_tiempo[['Fecha_ID', 'Fecha', 'Anio', 'Mes', 'Nombre_Mes', 'Numero_Semana', 'Trimestre', 'Estacionalidad_Factor']]


def generar_dim_tienda(num_tiendas):
    """ Crea la tabla de Dimensión Tienda e inyecta errores categóricos y de formato. """
    print("-> Generando Dim_Tienda...")
    dim_tienda = pd.DataFrame({'Tienda_ID': range(1, num_tiendas + 1)})
    
    regiones = ['Norte', 'Centro', 'Sur', 'Occidente']
    formatos = ['Supermercado', 'Tienda_Especializada', 'Tienda_Conveniencia']
    
    dim_tienda['Region'] = np.random.choice(regiones, num_tiendas, p=[0.25, 0.35, 0.20, 0.20])
    dim_tienda['Formato_Tienda'] = np.random.choice(formatos, num_tiendas, p=[0.5, 0.3, 0.2])
    dim_tienda['Atractivo_Factor'] = np.random.uniform(0.5, 1.5, num_tiendas).round(2)
    
    # PROBLEMA 1: Inconsistencia en Región (Mayúsculas, tildes, espacios extra)
    sample_size = int(len(dim_tienda) * INCONSISTENCY_RATIO)
    inconsistency_indices = np.random.choice(dim_tienda.index, size=sample_size, replace=False)
    
    for idx in inconsistency_indices:
        original_name = dim_tienda.loc[idx, 'Region']
        if random.random() < 0.3:
            dim_tienda.loc[idx, 'Region'] = original_name.upper() # Todo Mayúsculas
        elif random.random() < 0.6:
            dim_tienda.loc[idx, 'Region'] = original_name.replace('e', 'é') # Tildes incorrectas
        else:
            dim_tienda.loc[idx, 'Region'] = original_name + ' ' # Espacio extra
            
    # PROBLEMA 2: Inconsistencia en Formato_Tienda (Nombre alternativo)
    sample_size_format = int(len(dim_tienda) * INCONSISTENCY_RATIO)
    inconsistency_format_indices = np.random.choice(dim_tienda.index, size=sample_size_format, replace=False)
    for idx in inconsistency_format_indices:
        if dim_tienda.loc[idx, 'Formato_Tienda'] == 'Tienda_Conveniencia':
            dim_tienda.loc[idx, 'Formato_Tienda'] = 'MiniMarket'
            
    return dim_tienda[['Tienda_ID', 'Region', 'Formato_Tienda', 'Atractivo_Factor']]


def generar_dim_producto(num_productos):
    """ Crea la tabla de Dimensión Producto e inyecta errores de formato y outliers. """
    print("-> Generando Dim_Producto...")
    dim_producto = pd.DataFrame({'Producto_ID': range(1001, 1001 + num_productos)})
    
    marcas = ['Acuavida', 'SaborMax', 'UltraFizz', 'Vitality', 'Frescura']
    segmentos = ['Agua', 'Gaseosa', 'Jugo', 'Energetica']
    sabores = ['Original', 'Naranja', 'Limon', 'Manzana', 'Uva', 'Fresa']
    envases = ['Botella_PET', 'Lata', 'Carton', 'Vidrio']
    tamanos = ['250ml', '500ml', '1L', '2L']
    
    dim_producto['Marca'] = np.random.choice(marcas, num_productos)
    dim_producto['Segmento'] = np.random.choice(segmentos, num_productos)
    dim_producto['Sabor'] = np.random.choice(sabores, num_productos)
    dim_producto['Tipo_Envase'] = np.random.choice(envases, num_productos)
    dim_producto['Tamano'] = np.random.choice(tamanos, num_productos)
    dim_producto['Precio_Base'] = np.random.uniform(0.5, 5.0, num_productos).round(2)
    dim_producto['Demanda_Factor'] = np.random.uniform(0.8, 1.2, num_productos).round(2)
    
    # PROBLEMA 1: Outliers en Precio_Base (Valores negativos o cero)
    sample_size = int(len(dim_producto) * 0.001) # 2% de outliers
    outlier_indices = np.random.choice(dim_producto.index, size=sample_size, replace=False)
    dim_producto.loc[outlier_indices, 'Precio_Base'] = np.random.uniform(-0.5, 0.0, sample_size).round(2)
    
    # PROBLEMA 2: Nombres de Marca con inconsistencias (acentos)
    sample_size_marca = int(len(dim_producto) * INCONSISTENCY_RATIO)
    inconsistency_marca_indices = np.random.choice(dim_producto.index, size=sample_size_marca, replace=False)
    for idx in inconsistency_marca_indices:
        if dim_producto.loc[idx, 'Marca'] == 'Acuavida':
            dim_producto.loc[idx, 'Marca'] = 'AcuaVida' # Inconsistencia de capitalización
            
    return dim_producto[['Producto_ID', 'Marca', 'Segmento', 'Sabor', 'Tipo_Envase', 'Tamano', 'Precio_Base', 'Demanda_Factor']]


def generar_fact_ventas(dim_tiempo, dim_producto, dim_tienda, num_rows):
    """ 
    Crea la tabla de Hechos (Fact_Ventas) e inyecta valores nulos y ceros en métricas. 
    Garantiza que haya ventas en todos los meses del año.
    """
    print("-> Generando Fact_Ventas (ventas distribuidas equitativamente durante el año)...")
    
    # --- Asegurar cobertura uniforme de fechas ---
    fechas_validas = dim_tiempo.loc[dim_tiempo['Fecha_ID'].astype(str).str.isnumeric(), 'Fecha_ID']
    meses = dim_tiempo['Mes'].unique()
    num_meses = len(meses)

    # Repartimos las filas aproximadamente equitativas entre meses
    filas_por_mes = num_rows // num_meses
    filas_restantes = num_rows - (filas_por_mes * num_meses)
    
    fact_list = []
    
    for i, mes in enumerate(sorted(meses)):
        # Fechas de ese mes
        fechas_mes = fechas_validas[dim_tiempo['Mes'] == mes]
        if fechas_mes.empty:
            continue
        
        # Cuántas filas asignar a este mes
        n = filas_por_mes + (1 if i < filas_restantes else 0)
        
        # Muestreo de fechas dentro del mes
        fecha_ids = np.random.choice(fechas_mes, size=n, replace=True)
        tienda_ids = np.random.choice(dim_tienda['Tienda_ID'], size=n)
        producto_ids = np.random.choice(dim_producto['Producto_ID'], size=n)
        
        fact_mes = pd.DataFrame({
            'Tienda_ID': tienda_ids,
            'Producto_ID': producto_ids,
            'Fecha_ID': fecha_ids,
            'Distribucion_Numerica': np.random.uniform(0.5, 1.0, n).round(2),
            'Distribucion_Ponderada': np.random.uniform(0.6, 1.0, n).round(2)
        })
        fact_list.append(fact_mes)
    
    # Combinar todos los meses y mezclar
    fact_ventas = pd.concat(fact_list, ignore_index=True)
    fact_ventas = fact_ventas.sample(frac=1, random_state=42).reset_index(drop=True)
    
    # --- Mapeos de factores ---
    temp_tienda_map = dim_tienda.set_index('Tienda_ID')['Atractivo_Factor']
    temp_producto_map = dim_producto.set_index('Producto_ID')[['Demanda_Factor', 'Precio_Base']]

    fact_ventas['Atractivo_Tienda'] = fact_ventas['Tienda_ID'].map(temp_tienda_map)
    fact_ventas['Demanda_Factor'] = fact_ventas['Producto_ID'].map(temp_producto_map['Demanda_Factor'])
    fact_ventas['Precio_Base'] = fact_ventas['Producto_ID'].map(temp_producto_map['Precio_Base'])
    
    # Precio de venta
    fact_ventas['Precio'] = (fact_ventas['Precio_Base'] * np.random.uniform(0.9, 1.1, len(fact_ventas))).round(2)
    
    # Estacionalidad
    def safe_extract_month_day(fecha_id_series):
        fecha_id_str = fecha_id_series.astype(str).str.slice(4, 8)
        month_day_int = pd.to_numeric(fecha_id_str, errors='coerce').fillna(0).astype(int)
        estacionalidad = np.cos((month_day_int - 601) * 2 * np.pi / 1200)
        estacionalidad[month_day_int == 0] = 1.0 
        return estacionalidad
        
    fact_ventas['Estacionalidad_Factor'] = safe_extract_month_day(fact_ventas['Fecha_ID'])
    
    # Calcular volumen y valor
    fact_ventas['Out_Of_Stock_Flag'] = np.random.choice([0, 1], len(fact_ventas), p=[0.95, 0.05])
    base_volume = (
        fact_ventas['Demanda_Factor'] * fact_ventas['Atractivo_Tienda'] * 
        fact_ventas['Estacionalidad_Factor'] * np.random.uniform(80, 120, len(fact_ventas))
    )
    fact_ventas['Ventas_Volumen'] = base_volume.astype(int)
    fact_ventas['Ventas_Valor'] = (fact_ventas['Ventas_Volumen'] * fact_ventas['Precio']).round(2)

    # --- Inyección de errores controlados ---
    sample_size_null = int(len(fact_ventas) * INCONSISTENCY_RATIO)
    null_indices = np.random.choice(fact_ventas.index, size=sample_size_null, replace=False)
    fact_ventas.loc[null_indices, 'Precio'] = np.nan
    print(f"   - Inyectados {sample_size_null} valores nulos en Precio.")
    
    sample_size_zero = int(len(fact_ventas) * INCONSISTENCY_RATIO)
    zero_indices = np.random.choice(fact_ventas.index, size=sample_size_zero, replace=False)
    fact_ventas.loc[zero_indices, 'Ventas_Volumen'] = 0
    print(f"   - Inyectados {sample_size_zero} valores cero en Ventas_Volumen.")
    
    return fact_ventas[['Tienda_ID', 'Producto_ID', 'Fecha_ID', 'Distribucion_Numerica', 
                        'Distribucion_Ponderada', 'Precio', 'Out_Of_Stock_Flag', 
                        'Ventas_Volumen', 'Ventas_Valor']]


# ==============================================================================
# 2. Función de Exportación Principal
# ==============================================================================

def generar_datos_sucio(base_filename='datos_bi_sucio_bebidas', num_dias=730, num_tiendas=50, num_productos=100, num_rows_fact=1000000):
    """
    Genera y exporta todos los archivos CSV con datos sucios simulados.
    """
    start_date = '2023-01-01'
    end_date = pd.to_datetime(start_date) + pd.Timedelta(days=num_dias - 1)

    print(f"--- INICIANDO GENERADOR DE DATOS SUCIOS BI ---")
    print(f"Rango de fechas: {start_date} a {end_date.strftime('%Y-%m-%d')}")
    print(f"Filas de hechos (aprox): {num_rows_fact:,}")
    
    # Generación de dimensiones
    dim_tiempo = generar_dim_tiempo(start_date, end_date)
    dim_tienda = generar_dim_tienda(num_tiendas)
    dim_producto = generar_dim_producto(num_productos)
    
    # Generación de hechos
    fact_ventas = generar_fact_ventas(dim_tiempo, dim_producto, dim_tienda, num_rows_fact)
    
    # Exportación a CSV
    print("\n--- EXPORTANDO ARCHIVOS CSV ---")
    dataframes = {
        f'{base_filename}_dim_tiempo.csv': dim_tiempo,
        f'{base_filename}_dim_tienda.csv': dim_tienda,
        f'{base_filename}_dim_producto.csv': dim_producto,
        f'{base_filename}_fact_ventas.csv': fact_ventas,
    }
    
    for filename, df in dataframes.items():
        # Nota: La exportación de 40M de filas a CSV puede ser muy lenta y generar un archivo grande.
        df.to_csv(filename, index=False, encoding='utf-8')
        print(f"✅ Guardado: {filename} ({len(df):,} filas)")

    print("--- GENERACIÓN DE DATOS COMPLETADA ---")

if __name__ == "__main__":
    # ¡Ajuste para 40 millones de registros de hechos!
    generar_datos_sucio(
        base_filename='datos_bi_sucio_bebidas_40M',
        num_dias=730,
        num_tiendas=50,
        num_productos=100,
        num_rows_fact=40000000 
    )


--- INICIANDO GENERADOR DE DATOS SUCIOS BI ---
Rango de fechas: 2023-01-01 a 2024-12-30
Filas de hechos (aprox): 40,000,000
-> Generando Dim_Tiempo...
   - Inyectados 0 valores nulos en Fecha_ID.
   - Inyectados 1 errores de formato en Fecha_ID.
-> Generando Dim_Tienda...
-> Generando Dim_Producto...
-> Generando Fact_Ventas (ventas distribuidas equitativamente durante el año)...
   - Inyectados 100000 valores nulos en Precio.
   - Inyectados 100000 valores cero en Ventas_Volumen.

--- EXPORTANDO ARCHIVOS CSV ---
✅ Guardado: datos_bi_sucio_bebidas_40M_dim_tiempo.csv (730 filas)
✅ Guardado: datos_bi_sucio_bebidas_40M_dim_tienda.csv (50 filas)
✅ Guardado: datos_bi_sucio_bebidas_40M_dim_producto.csv (100 filas)
✅ Guardado: datos_bi_sucio_bebidas_40M_fact_ventas.csv (40,000,000 filas)
--- GENERACIÓN DE DATOS COMPLETADA ---
